# Test Diff

In [1]:
import sys
!{sys.executable} -m pip install stixorm
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, Incident, URL, EmailMessage, ObservedData
)
from stixorm.module.definitions.os_threat import (
    EventCoreExt, Event, SocialMediaContact, ContactNumber, IncidentCoreExt, TaskCoreExt,
    Task, SightingEvidence, Sequence, SequenceExt, AnecdoteExt, Anecdote,
    SightingAnecdote, SightingAlert, SightingContext, SightingExclusion,
    SightingEnrichment, SightingHunt, SightingFramework, SightingExternal
)
from stixorm.module.authorise import import_type_factory
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects
from deepdiff import DeepDiff, parse_path
from pprint import pprint
import_type = import_type_factory.get_all_imports()
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
import json

# Setup Context Memory Variables

In [2]:

# from Utilities.local_make_general import invoke_context_get_block, invoke_context_save_block

# Common File Stuff
TR_Common_Files = "./generated/os-triage/common_files"
common = [
    {"module": "convert_n_and_e", "file": "convert_n_and_e.py", "url" : "https://raw.githubusercontent.com/typerefinery-ai/brett_blocks/main/Block_Families/General/_library/convert_n_and_e.py"}
]

# OS_Triage Memory Stuff
TR_Context_Memory_Dir = "./generated/os-triage/context_mem"
TR_User_Dir = "/usr"
context_map = "context_map.json"
user_data = {
    "global": "/global_variables_dict.json",
    "me": "/cache_me.json",
    "team": "/cache_team.json",
    "relations" : "/relations.json",
    "edges" : "/edges.json",
    "relation_edges" : "/relation_edges.json",
    "relation_replacement_edges" : "/relation_replacement_edges.json"
}
comp_data = {
    "users": "/users.json",
    "company" : "/company.json",
    "assets" : "/assets.json",
    "systems" : "/systems.json",
    "relations" : "/relations.json",
    "edges" : "/edges.json",
    "relation_edges" : "/relation_edges.json",
    "relation_replacement_edges" : "/relation_replacement_edges.json"
}
incident_data = {
    "incident" : "/incident.json",
    "start" : "/sequence_start_refs.json",
    "sequence" : "/sequence_refs.json",
    "impact" : "/impact_refs.json",
    "event" : "/event_refs.json",
    "task" : "/task_refs.json",
    "other" : "/other_object_refs.json",
    "unattached" : "/unattached_objs.json",
    "unattached_relations" : "/unattached_relation.json",
    "relations" : "/incident_relations.json",
    "edges" : "/incident_edges.json",
    "relation_edges" : "/relation_edges.json",
    "relation_replacement_edges" : "/relation_replacement_edges.json"
}
field_names = {
    "start" : "sequence_start_refs",
    "sequence" : "sequence_refs",
    "impact" : "impact_refs",
    "event" : "event_refs",
    "task" : "task_refs",
    "other" : "other_object_refs"
}


# Lets run some tests

In [3]:
A = {0, 2, 4, 6, 8}
B = {1, 2, 3, 4, 5}
C = A - B
print(f"Delete -> Things in old, not in new ->{C}")
D = B - A
print(f"Add -> Things in new, not in old ->{D}")
E = A & B
print(f"Intersections -> Things which may have changed ->{E}")


Delete -> Things in old, not in new ->{0, 8, 6}
Add -> Things in new, not in old ->{1, 3, 5}
Intersections -> Things which may have changed ->{2, 4}


In [4]:
from datetime import datetime
created ="2024-01-24T12:38:07.927748Z"
returned = "2024-01-24T12:38:07.927Z"
cr_date = datetime.strptime(created, "%Y-%m-%dT%H:%M:%S.%fZ")
ret_date = datetime.strptime(returned, "%Y-%m-%dT%H:%M:%S.%fZ")
print(type(cr_date))
print(cr_date)
cr_date_String = str(cr_date.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3])
print(cr_date_String)
print(cr_date.microsecond)
microsecs = ret_date.microsecond
print(type(microsecs))
milisecs = (round(microsecs/1000))
print(milisecs)
dt = created.split('.')
print(dt)
actual = dt[0] + "." + str(milisecs) + "Z"
print(actual)




<class 'datetime.datetime'>
2024-01-24 12:38:07.927748
2024-01-24T12:38:07.927
927748
<class 'int'>
927
['2024-01-24T12:38:07', '927748Z']
2024-01-24T12:38:07.927Z


## setup details in typedb

First load details into TypeDB, and then get them back

In [5]:
from stixorm.module.typedb import TypeDBSink, TypeDBSource
from stixorm.module.authorise import import_type_factory
from Utilities.local_make_sdo import invoke_make_identity_block
from Utilities.util import emulate_ports, unwind_ports, conv
connection = {
    "uri": "localhost",
    "port": "1729",
    "database": "stix",
    "user": None,
    "password": None
}

object_base = "../Block_Families/OS_Triage/"
import_type = import_type_factory.get_all_imports()
typedb_in_path = "Update_Context/data_in_typedb.json"
updated_list_path = "Update_Context/data_new_in_os_triage.json"
updated_list_path2 = "Update_Context/data_new_in_os_triage_v2.json"

#
# Open the data for typedb
#



typedb_sink = TypeDBSink(connection=connection, clear=True, import_type=import_type)
typedb_source = TypeDBSource(connection=connection,)
results_raw = typedb_sink.add(comp_ident)
ret_ident = typedb_source.get(comp_ident["id"])
print(ret_ident)

type identity->{'identity_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'name': 'Example Company', 'description': 'An Example Organisation', 'identity_class': 'organization', 'sectors': 'technology'}, 'extensions': {}, 'sub': {}}}
type identity-><class 'dict'>, type email-><class 'NoneType'>, type user acct-><class 'NoneType'>
Step 1 >>
Step 2 >>
Step 3 >>
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--c6cc716f-9712-4c9f-a71b-c4aa249f14a6",
    "created": "2024-10-14T00:33:09.985Z",
    "modified": "2024-10-14T00:33:09.985Z",
    "name": "Example Company",
    "description": "An Example Organisation",
    "identity_class": "organization",
    "sectors": [
        "technology"
    ]
}


NameError: name 'local_context' is not defined

In [ ]:
test = "20.20-10-19T01:01:.01Z"
ret = test.find(".")
print(ret)